***GENERATED CODE FOR loanpaymentsdataregg PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[Loan_ID]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[Loan_ID]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'xqd12160159', 'max': 'xqd91160291', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[L...'}, {'feature_label': 'Loan_payments_data_regg[Loan_ID]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[Loan_ID]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[loan_status]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[loan_status]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'COLLECTION', 'max': 'PAIDOFF', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[l...'}, {'feature_label': 'Loan_payments_data_regg[loan_status]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[loan_status]')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[effective_date]', 'transformation_label': 'Extract Date'}], 'feature': 'Loan_payments_data_regg[effective_date]', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Loan_payments_data_regg[e...'}, {'feature_label': 'Loan_payments_data_regg[effective_date]', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[effective_date]')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[due_date]', 'transformation_label': 'Extract Date'}], 'feature': 'Loan_payments_data_regg[due_date]', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Loan_payments_data_regg[d...'}, {'feature_label': 'Loan_payments_data_regg[due_date]', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[due_date]')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[paid_off_DateTime]', 'transformation_label': 'Extract Date'}], 'feature': 'Loan_payments_data_regg[paid_off_DateTime]', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '100'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Loan_payments_data_regg[p...'}, {'feature_label': 'Loan_payments_data_regg[paid_off_DateTime]', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[paid_off_DateTime]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[education]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[education]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Bechalor', 'max': 'college', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[e...'}, {'feature_label': 'Loan_payments_data_regg[education]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[education]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[Gender]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[Gender]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[G...'}, {'feature_label': 'Loan_payments_data_regg[Gender]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[Gender]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[effective_Month]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[effective_Month]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'September', 'max': 'September', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[e...'}, {'feature_label': 'Loan_payments_data_regg[effective_Month]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[effective_Month]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[due_Month]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[due_Month]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'November', 'max': 'September', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[d...'}, {'feature_label': 'Loan_payments_data_regg[due_Month]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[due_Month]')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[paid_off_Date]', 'transformation_label': 'Extract Date'}], 'feature': 'Loan_payments_data_regg[paid_off_Date]', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '100'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Loan_payments_data_regg[p...'}, {'feature_label': 'Loan_payments_data_regg[paid_off_Date]', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[paid_off_Date]')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_payments_data_regg[paid_off_Month]', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_payments_data_regg[paid_off_Month]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '400', 'mean': '', 'stddev': '', 'min': 'December', 'max': 'September', 'missing': '100', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_payments_data_regg[p...'}, {'feature_label': 'Loan_payments_data_regg[paid_off_Month]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Loan_payments_data_regg[paid_off_Month]')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run loanpaymentsdatareggHooks.ipynb
try:
	#sourcePreExecutionHook()

	loanpaymentsdataregghdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1676460116.csv', 'filename': 'tempFile1676460116.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(loanpaymentsdataregghdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run loanpaymentsdatareggHooks.ipynb
try:
	#transformationPreExecutionHook()

	loanpaymentsdatareggautofe = TransformationMain.run(loanpaymentsdataregghdfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Loan_payments_data_regg[Loan_ID]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[Loan_ID]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "xqd12160159", "max": "xqd91160291", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[L..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[loan_status]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[loan_status]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "COLLECTION", "max": "PAIDOFF", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[l..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[Principal]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "943.2", "stddev": "115.24", "min": "300", "max": "1000", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[P..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[terms]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "22.82", "stddev": "8.0", "min": "7", "max": "30", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[t..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[effective_date]", "transformation_label": "Extract Date"}], "feature": "Loan_payments_data_regg[effective_date]", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Loan_payments_data_regg[e..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[due_date]", "transformation_label": "Extract Date"}], "feature": "Loan_payments_data_regg[due_date]", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Loan_payments_data_regg[d..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[paid_off_DateTime]", "transformation_label": "Extract Date"}], "feature": "Loan_payments_data_regg[paid_off_DateTime]", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "100"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Loan_payments_data_regg[p..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[age]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "31.12", "stddev": "6.08", "min": "18", "max": "51", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[a..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[education]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[education]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bechalor", "max": "college", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[e..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[Gender]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[Gender]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[G..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[effective_Year]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2016.0", "stddev": "0.0", "min": "2016", "max": "2016", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[e..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[effective_Month]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[effective_Month]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "September", "max": "September", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[e..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[due_Year]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2016.0", "stddev": "0.0", "min": "2016", "max": "2016", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[d..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[due_Month]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[due_Month]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "November", "max": "September", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[d..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[paid_off_Date]", "transformation_label": "Extract Date"}], "feature": "Loan_payments_data_regg[paid_off_Date]", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "100"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Loan_payments_data_regg[p..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[paid_off_Year]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "400", "mean": "2016.0", "stddev": "0.0", "min": "2016.0", "max": "2016.0", "missing": "100"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[p..."}, {"transformationsData": [{"feature_label": "Loan_payments_data_regg[paid_off_Month]", "transformation_label": "String Indexer"}], "feature": "Loan_payments_data_regg[paid_off_Month]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "400", "mean": "", "stddev": "", "min": "December", "max": "September", "missing": "100", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[p..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[effective_MonthNumber]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.0", "stddev": "0.0", "min": "9", "max": "9", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[e..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[due_MonthNumber]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9.62", "stddev": "0.57", "min": "9", "max": "11", "missing": "0"}, "updatedLabel": "Loan_payments_data_regg[d..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[paid_off_MonthNumber]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "400", "mean": "9.6", "stddev": "0.65", "min": "9.0", "max": "12.0", "missing": "100"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[p..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Loan_payments_data_regg[past_due_days]", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "200", "mean": "36.01", "stddev": "29.38", "min": "1.0", "max": "76.0", "missing": "300"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Loan_payments_data_regg[p..."}]}))

	#transformationPostExecutionHook(loanpaymentsdatareggautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run loanpaymentsdatareggHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(loanpaymentsdatareggautofe, ["Loan_payments_data_regg[Principal]", "Loan_payments_data_regg[terms]", "Loan_payments_data_regg[age]", "Loan_payments_data_regg[effective_Year]", "Loan_payments_data_regg[due_Year]", "Loan_payments_data_regg[paid_off_Year]", "Loan_payments_data_regg[effective_MonthNumber]", "Loan_payments_data_regg[due_MonthNumber]", "Loan_payments_data_regg[paid_off_MonthNumber]", "Loan_payments_data_regg[past_due_days]", "Loan_payments_data_regg[loan_status]_stringindexer", "Loan_payments_data_regg[effective_date]_dayofmonth", "Loan_payments_data_regg[effective_date]_month", "Loan_payments_data_regg[effective_date]_year", "Loan_payments_data_regg[due_date]_dayofmonth", "Loan_payments_data_regg[due_date]_month", "Loan_payments_data_regg[due_date]_year", "Loan_payments_data_regg[paid_off_DateTime]_dayofmonth", "Loan_payments_data_regg[paid_off_DateTime]_month", "Loan_payments_data_regg[paid_off_DateTime]_year", "Loan_payments_data_regg[education]_stringindexer", "Loan_payments_data_regg[Gender]_stringindexer", "Loan_payments_data_regg[effective_Month]_stringindexer", "Loan_payments_data_regg[due_Month]_stringindexer", "Loan_payments_data_regg[paid_off_Date]_dayofmonth", "Loan_payments_data_regg[paid_off_Date]_month", "Loan_payments_data_regg[paid_off_Date]_year", "Loan_payments_data_regg[paid_off_Month]_stringindexer"], "Loan_payments_data_regg[Loan_ID]_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

